In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import sys 
import os
sys.path.append(os.path.abspath("/media/sf_Project2/Code"))
from IO import Input
from IO import Output
import pandas as pd

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [4]:
with cd("/media/sf_Project2/Code"): 
    train_dataset = np.array(Input.load_trainset_caffefeatures(featureSelectionMethod='RF',Percentile = 100)).astype('float32')
    train_labels = np.array(Input.load_trainset_labels()).astype('float32')
    valid_dataset = np.array(Input.load_validationset_caffefeatures(featureSelectionMethod='RF',Percentile = 100)).astype('float32')
    valid_labels = np.array(Input.load_validationset_labels()).astype('float32')

In [5]:
num_labels=10
train_labels = np.squeeze((np.arange(num_labels) == train_labels[:,None]).astype(np.float32))
valid_labels = np.squeeze((np.arange(num_labels) == valid_labels[:,None]).astype(np.float32))
train_labels0 = train_labels[:,1]
train_labels0 = train_labels0.reshape((train_labels.shape[0],1))

print(train_labels0)
#print(train_dataset)
#train_labels = train_labels.reshape((train_labels.shape[0],1))
#valid_labels = valid_labels.reshape((valid_labels.shape[0],1))
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
feature_size = train_dataset.shape[1]
print(train_labels)

[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
Training set (14363, 3983) (14363, 10)
Validation set (8061, 3983) (8061, 10)
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels,1))
          / predictions.shape[0])

# Network:

In [7]:
batch_size = 64
hlSize0 = 516
beta = 0.004
#decay_steps = 200
#decay_rate = 0.90
#learningStart=0.0007
decay_steps = 180
decay_rate = 0.98
learningStart=0.00009

stdv = 0.03
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=stdv))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize0,num_labels], 
            stddev=stdv))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))

  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.matmul(layer1, layer1_weights) + layer1_biases
    return layer2
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(input_weights) + \
    beta * tf.nn.l2_loss(layer1_weights)
  #  beta * tf.nn.l2_loss(layer2_weights) + \
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  learning_rate = tf.train.exponential_decay(learningStart, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")
  input_weights_val = input_weights.eval()
  input_biases_val = input_biases.eval()
  layer1_weights_val = layer1_weights.eval()
  layer1_biases_val = layer1_biases.eval()
  valid_prediction_val = valid_prediction.eval()

Initialized
Minibatch loss at step 0: 6.242658
Minibatch accuracy: 0.0%
Validation accuracy: 8.4%
Minibatch loss at step 100: 5.136612
Minibatch accuracy: 3.1%
Validation accuracy: 12.3%
Minibatch loss at step 200: 4.925267
Minibatch accuracy: 32.8%
Validation accuracy: 13.4%
Minibatch loss at step 300: 5.418179
Minibatch accuracy: 0.0%
Validation accuracy: 12.5%
Minibatch loss at step 400: 5.389382
Minibatch accuracy: 0.0%
Validation accuracy: 16.9%
Minibatch loss at step 500: 5.406830
Minibatch accuracy: 0.0%
Validation accuracy: 14.4%
Minibatch loss at step 600: 5.117654
Minibatch accuracy: 10.9%
Validation accuracy: 18.9%
Minibatch loss at step 700: 4.680746
Minibatch accuracy: 57.8%
Validation accuracy: 18.0%
Minibatch loss at step 800: 5.033174
Minibatch accuracy: 9.4%
Validation accuracy: 20.2%
Minibatch loss at step 900: 4.758352
Minibatch accuracy: 43.8%
Validation accuracy: 20.7%
Minibatch loss at step 1000: 4.707358
Minibatch accuracy: 57.8%
Validation accuracy: 22.6%
Miniba

In [9]:
validData = pd.DataFrame(valid_prediction_val)
Output.to_outputfile(validData,1,'NNSTRUCTURE7valid')

In [10]:
with cd("/media/sf_Project2/Code"): 
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(30000),'RF',100)).astype('float32')

In [11]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    firstHalfTest = tf.nn.softmax(layer2).eval()
print(firstHalfTest)

[[ 0.06329893  0.32748848  0.06300098 ...,  0.03864574  0.04600954
   0.13266549]
 [ 0.24048705  0.08168793  0.05648522 ...,  0.02412184  0.05482405
   0.0986474 ]
 [ 0.1110265   0.04528941  0.30981088 ...,  0.03188354  0.11247267
   0.23245654]
 ..., 
 [ 0.12895267  0.09142771  0.04086019 ...,  0.08836001  0.09279449
   0.12798877]
 [ 0.27548581  0.1776623   0.09167734 ...,  0.05189721  0.02843883
   0.03427691]
 [ 0.07029021  0.18386991  0.13908726 ...,  0.0250117   0.05912545
   0.013356  ]]


In [12]:
with cd("/media/sf_Project2/Code"): 
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(30000,60000),'RF',100)).astype('float32')

In [13]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    secondHalfTest = tf.nn.softmax(layer2).eval()
print(secondHalfTest)

[[ 0.04254227  0.13280852  0.07062809 ...,  0.074274    0.01857557
   0.09168234]
 [ 0.30529121  0.04095709  0.0151107  ...,  0.02018827  0.02076059
   0.06663001]
 [ 0.09787036  0.26003116  0.20167604 ...,  0.03652278  0.08004014
   0.03488561]
 ..., 
 [ 0.30568621  0.06550854  0.0123859  ...,  0.12942395  0.04053192
   0.1380109 ]
 [ 0.31968117  0.011182    0.00976147 ...,  0.17636795  0.02771321
   0.23987441]
 [ 0.16726017  0.07674735  0.06646004 ...,  0.01434362  0.11277219
   0.14769414]]


In [14]:
with cd("/media/sf_Project2/Code"): 
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(60000,80000),'RF',100)).astype('float32')

In [15]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    thirdHalfTest = tf.nn.softmax(layer2).eval()
print(thirdHalfTest)

[[ 0.0842452   0.05863297  0.08064885 ...,  0.07276397  0.05425767
   0.07034339]
 [ 0.05660829  0.05322461  0.03187521 ...,  0.19003496  0.05924417
   0.12862273]
 [ 0.18723793  0.04147238  0.05337696 ...,  0.07124321  0.04266255
   0.16713482]
 ..., 
 [ 0.05908116  0.01816196  0.01661102 ...,  0.02768991  0.02275595
   0.08630345]
 [ 0.03613416  0.03610254  0.06415363 ...,  0.37308347  0.11508513
   0.04020213]
 [ 0.22548842  0.0365442   0.02960149 ...,  0.01616046  0.05358925
   0.11068434]]


In [17]:
testClass = np.concatenate([firstHalfTest,secondHalfTest,thirdHalfTest],0)
testClass = pd.DataFrame(testClass)
Output.to_outputfile(testClass,1,"NNSTRUCTURE7testset",validation=False)